In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean


In [6]:
pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [7]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [8]:
sql1 = '''
SELECT 
tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.refund_date '预计还款日期', tt.reality_refund_date '实际还款日期', 
tt.money '应付金额', (tt.real_pay_money+tt.pay_discount_money) '实付金额',tt.pay_discount_money, tt.discount_all_money,tt.min_date, tt.max_date, tt.diff_date
FROM (
SELECT  t.order_number,t.true_name,t.user_mobile,t.product_name,t.create_time, t.refund_date,t.reality_refund_date,t.money,t.real_pay_money,t.min_date,
case when max_date>=DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY) then CURRENT_DATE else t.max_date END max_date,
case when t.min_date=t.max_date then DATEDIFF(CURRENT_DATE, t.min_date) 
	        when max_date>CURRENT_DATE then DATEDIFF(CURRENT_DATE, t.min_date) 
			else DATEDIFF(t.max_date, t.min_date) END diff_date
,t.pay_discount_money, t.discount_all_money
FROM (
SELECT -- 查询总体订单和去重以及剔除商家 
om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money,tos.pay_discount_money, tos.discount_all_money
,min(tos.refund_date)over(partition by om.order_number) min_date
,max(tos.refund_date)over(partition by om.order_number) max_date
FROM db_digua_business.t_postlease_receivables_monitoring   tprm 
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND tos.refund_date = DATE_ADD(CURRENT_DATE,INTERVAL +1 DAY)
AND (tos.real_pay_money+tos.pay_discount_money)<(tos.money+tos.sesame_promo_money)
and tos.reality_refund_date is null
and pa.type!=4
and om.has_actual!=1
) t
) tt
WHERE diff_date<=0 and refund_date>=CURRENT_DATE and refund_date<=DATE_ADD(CURRENT_DATE, INTERVAL -0 DAY);
'''
df_order = query(sql1)
df_order.shape

(119, 14)

In [9]:
sql2 = '''
SELECT tt.order_number '订单号', tt.true_name '姓名', tt.user_mobile '手机号', tt.product_name '产品名称', tt.create_time '下单日期', tt.relet_start_time '预计还款日期', tt.money '应付金额', tt.min_time FROM (
SELECT t.order_number, t.true_name, t.user_mobile, t.product_name, t.create_time, t.relet_start_time, t.money,
min(relet_start_time)over(partition by t.order_id) min_time FROM(
SELECT om.order_number, tor.order_id, tmu.true_name, om.user_mobile, tod.product_name, om.create_time, tor.relet_start_time, tor.money 
FROM db_digua_business.t_postlease_receivables_monitoring   tprm
LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_relet tor on tprm.order_id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
and cycle_status = 1
and tprm.activity_id != 335
and om.has_actual!=1
) t
) tt
WHERE min_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +1 DAY)) and relet_start_time in (CURRENT_DATE, DATE_ADD(CURRENT_DATE, INTERVAL +0 DAY))
'''
df_order2 = query(sql2)
df_order2.shape

(15, 8)

In [10]:
sql3 = '''
SELECT yoo.order_number '订单号', tmu.true_name '姓名', om.user_mobile '手机号', tod.product_name '产品名称', om.create_time '下单日期', yoo.overdue_time '预计还款日期', yoo.overdue_money '应付金额'
FROM db_digua_business.t_order om
LEFT JOIN db_rent.ya_order_overdue yoo on yoo.order_id=om.id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
WHERE DATE_FORMAT(om.create_time,'%Y-%m')=DATE_FORMAT(DATE_ADD(CURRENT_DATE,INTERVAL -1 MONTH), '%Y-%m')  and yoo.type = 0
'''
df_order3 = query(sql3)
df_order3.shape

(6, 7)

In [11]:
sql4 = '''
SELECT  t.channel_name ,t.activity_name, t.total_describes, t.rejected, t.result, t.order_id,
t.order_number '订单号', t.true_name '姓名', t.user_mobile '手机号', t.product_name '产品名称', t.create_time '下单日期', t.refund_date '预计还款日期', t.reality_refund_date '实际还款日期', 
t.money '应付金额', t.real_pay_money '实付金额', diff_date '逾期天数',t.sort,t.order_method, t.status2, t.channel_type_id, t.trace_id
FROM (
SELECT -- 查询总体订单和去重以及剔除商家
cc.`name` as channel_name
,pa.`name` as activity_name, tprm.order_id
,om.order_number,tmu.true_name,om.user_mobile,tod.product_name,om.create_time,tos.refund_date,tos.reality_refund_date,tos.money,tos.real_pay_money, tos.sort,
case 	when tos.reality_refund_date is null then DATEDIFF(CURRENT_DATE, tos.refund_date)
			when DATE_FORMAT(tos.reality_refund_date,'%Y-%m-%d')>DATE_FORMAT(tos.refund_date,'%Y-%m-%d') then DATEDIFF(tos.reality_refund_date,tos.refund_date)
			else null END diff_date
,om.order_method,top.total_describes
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tori.decision_result) THEN JSON_EXTRACT(tori.decision_result, '$.trace_id') end,'"','') as trace_id 
,cc.channel_type_id
FROM  db_digua_business.t_order om
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring   tprm on tprm.order_id = om.id 
LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = tprm.merchant_id
LEFT JOIN db_digua_business.t_order_stages tos on om.id=tos.order_id
-- LEFT JOIN db_digua_business.t_order_relet tor on om.id=tor.order_id
LEFT JOIN db_digua_business.t_order_details tod on tod.order_id=om.id 
LEFT JOIN db_digua_business.t_member_user tmu on tmu.mobile=om.user_mobile
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_risk tori on om.id = tori.order_id
WHERE om.`status` in (2, 3, 4) 
AND tmer.shop_type !=2  -- 剔除企业 1、自营店铺；2、入驻店铺；3、代运营店铺'
AND tprm.merchant_id  not in (15,99)  -- 剔除CPS渠道合作 汇客好租
AND tprm.model_number not like '%探路者%'
-- AND DATE_FORMAT(om.create_time,'%Y-%m')>=DATE_FORMAT(DATE_ADD(CURRENT_DATE, INTERVAL -3 MONTH), '%Y-%m')
-- AND DATE_FORMAT(om.create_time,'%Y-%m')<=DATE_FORMAT(DATE_ADD(CURRENT_DATE, INTERVAL -1 MONTH), '%Y-%m')
AND tos.refund_date <= CURRENT_DATE
and pa.type != 4
) t
-- WHERE DATE_FORMAT(t.refund_date,'%Y-%m')=DATE_FORMAT(CURRENT_DATE,'%Y-%m')
'''
df_order4 = query(sql4)
df_order4.shape

(194685, 21)

In [12]:
dd_date = '2024-05'
sql5 = f'''
SELECT 
tprm.order_number '订单号', tprm.total_receivable '总应收', tprm.all_rental '总租金', tprm.all_deposit '总押金', tod.new_actual_money '买断价', (tod.new_actual_money-tprm.all_rental) '买断尾款', tprm.commodity_premium '溢价费', tprm.buyout_balance_payment '残值', om.user_mobile '手机号', tprm.search_time '下单月份', tprm.model_number '型号', tprm.specification '规格', tprm.order_create_time '下单时间', om.create_time '创建时间', tprm.rembursement_status '还款状态', 
tprm.paid_periods '已还期数', tprm.end_date '租期结束日期', tprm.order_finish_date '订单完成时间'
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.order_method, om.order_type, cc.channel_type_id
FROM db_digua_business.t_postlease_receivables_monitoring tprm
LEFT JOIN db_digua_business.t_order om ON tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE tprm.search_time='{dd_date}'
and tprm.platform_package != '租完即送'
and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124, 171, 172)
'''

df_order5 = query(sql5)
df_order5.shape

(1184, 23)

In [13]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r from db_credit.risk
'''
df_risk = query(sql_risk)

In [14]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [15]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

# 合并数据

In [16]:
df = df_order.copy()
df2 = df_order2.copy()
df3 = df_order3.copy()
df4 = df_order4.copy()
df5 = df_order5.copy()
# df.order_id.notnull().sum()

In [17]:

df4["取消原因"]=df4["total_describes"].str.split("客户申请取消：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["电审拒绝原因"]=df4["total_describes"].str.split("审核不通过：").str[1].astype(str).str.split("$").str[0].str.strip()
df4["无法联系原因"]=df4["total_describes"].str.split("用户无法联系：").str[1].astype(str).astype(str).str.split("$").str[0].str.strip()


df4.loc[:,"进件"]=np.where((df4["status2"]=="待支付")|(df4["status2"]=="订单取消"),"未进件","进件")
df4["进件"].value_counts()

# %%
df4.loc[:,"来源渠道"]=df4["channel_name"].fillna("未知渠道")

# 渠道归属

In [18]:
df4['拒绝理由'] = df4["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

In [19]:
df4.loc[:,"归属渠道"]=df4.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

TypeError: Data_Clean.qudao_type() missing 1 required positional argument: 'e'

In [71]:
# note 2025-02-24添加
df4 = df4.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')

# 判断 前置拦截   机审强拒   出库前风控强拒
df4['是否前置拦截'] = np.where((df4.result.str.contains('id_card不得为空') )|(df4.result.str.contains('mobile校验不通过')) |(df4.result.str.contains('name校验不通过'))|(df4.result.str.contains('年龄超过49岁或低于18岁'))|(df4.result=='风控未通过')|(df4.result=='黑名单用户'),1,0)
# df4['是否机审强拒'] = np.where((df4.result.str.contains('风控拒绝') & (~df4.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df4['是否出库前风控强拒'] = np.where((df4.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df4.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df4.result.str.contains('黑名单用户').fillna(False))) ,1,0)
df4['是否机审强拒'] = np.where(df4.status_r=='1', 1, 0)
df4['是否出库前风控强拒'] = np.where((df4.status_re=='1')|(df4.status_ra=='1'), 1, 0)


In [72]:
# 定义审核状态
df4.loc[:,"审核状态"]=df4.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df4["审核状态"].value_counts()  

审核状态
客户取消    184805
Name: count, dtype: int64

In [73]:

df4_j = df4[df4["进件"]=="进件"]
df4["是否进件"] = np.where(df4["进件"]=='进件',1,0)

In [74]:
df_all = pd.concat([df, df2])
df_all
# indx = round(len(df_all)/10)
# df_cs = df_all[:indx]
# df_xs = df_all
# df_all

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,应付金额,实付金额,pay_discount_money,discount_all_money,min_date,max_date,diff_date,min_time
0,A2024041417114321,赖玺如,13398144477,iPhone15ProMax 全新国行 顺丰包邮,2024-04-14 17:11:44,2025-03-17,None,1063.94,0.0,0.0,0.0,2025-03-17,2025-03-17,0.0,NaT
1,A2024051409384755,侯艳玲,15092663583,iPhone15Pro 全新国行 非监管机 顺丰包邮,2024-05-14 09:38:47,2025-03-17,None,443.38,0.0,0.0,0.0,2025-03-17,2025-03-17,0.0,NaT
2,A20240514165610158,顾光明,13629676924,华为Mate60Pro 旗舰手机 全新国行 顺丰包邮,2024-05-14 16:56:11,2025-03-17,None,466.36,0.0,0.0,0.0,2025-03-17,2025-03-17,0.0,NaT
3,A2024061400555690,梁凯,19538769079,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-06-14 00:55:56,2025-03-17,None,356.15,0.0,0.0,0.0,2025-03-17,2025-03-17,0.0,NaT
4,A2024061401183231,宋泽宇,18030843760,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,2024-06-14 01:18:32,2025-03-17,None,356.15,0.0,0.0,0.0,2025-03-17,2025-03-17,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,A2024011515080225,蔺国庆,18262388292,iPhone15ProMax 全新国行 顺丰包邮,2024-01-15 15:08:03,2025-03-17,NaN,665.47,NaN,NaN,NaN,NaT,NaT,NaN,2025-03-17
16,A2024011515170847,郭家杰,13414673023,iPhone15ProMax 全新国行 顺丰包邮,2024-01-15 15:17:09,2025-03-17,NaN,665.47,NaN,NaN,NaN,NaT,NaT,NaN,2025-03-17
17,A2024031214405753,林海,17767977704,iPhone15ProMax 全新国行 顺丰包邮,2024-03-12 14:40:58,2025-03-17,NaN,669.63,NaN,NaN,NaN,NaT,NaT,NaN,2025-03-17
18,A2024031311552666,马灿辉,17585112743,iPhone15ProMax 全新国行 顺丰包邮,2024-03-13 11:55:26,2025-03-17,NaN,669.63,NaN,NaN,NaN,NaT,NaT,NaN,2025-03-17


In [75]:
now_date = datetime.now().strftime('%Y-%m-%d')
next_date = (pd.to_datetime(now_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
# df_xs1 = df_xs[df_xs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2 = df_xs[df_xs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_xs2
df_xs = df_all[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].drop_duplicates().sort_values(by='预计还款日期')

In [76]:
# df_cs1 = df_cs[df_cs['预计还款日期']==now_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs2 = df_cs[df_cs['预计还款日期']==next_date][['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']]
# df_cs1
# df_cs = df_cs[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '应付金额']].sort_values(by='预计还款日期')
# df_cs

# 逾期订单天数

In [77]:
# df3['逾期天数'] = datetime.now() - df3['预计还款日期']
# df3
df3['逾期天数'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))- df3['预计还款日期']
# df3.groupby('逾期天数').agg({'订单号': 'count'})
# df3

# 下单时间上月截止至今日的订单

## 到期订单

In [78]:
df4['下单月份'] = df4['下单日期'].dt.strftime('%Y-%m')
df4['预计还款月份'] = df4['预计还款日期'].dt.strftime('%Y-%m')
create_month = df4['下单月份'].to_list()
# create_month

## 首逾

In [79]:
Today1 = datetime.now().strftime('%Y-%m-%d')
def order_end(df, month1, month2, keys):
    date_diff = (pd.to_datetime(Today1).to_period('M')-pd.to_datetime(month2).to_period('M')).n
    td = Today1
    df4_dq =df
    if date_diff==0:
        td
    elif date_diff==1:
        td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))).strftime('%Y-%m-%d')
    else:
        td = (pd.to_datetime(Today1)-timedelta(days=int(datetime.now().strftime('%d')))-pd.DateOffset(months=date_diff-1)).strftime('%Y-%m-%d')
    if keys == '首逾':
        df4_dq = df[(df['下单月份']==month1)&(df['预计还款月份']==month2)&(df['sort']==2)]
    elif keys == '历史逾期':
        df4_dq = df[df['预计还款月份']==month2]
    # 月总订单
    dd_dq_month = df4_dq['订单号'].count()
    # 逾期1+订单
    df_yq_1= df4_dq[(df4_dq['逾期天数']>=1)]
    dd_yq_1 = df_yq_1['订单号'].count()
    # 逾期4+订单
    df_yq_4= df4_dq[(df4_dq['逾期天数']>=4)]
    dd_yq_4 = df_yq_4['订单号'].count()
    # 逾期7+订单
    df_yq_7= df4_dq[(df4_dq['逾期天数']>=7)]
    dd_yq_7 = df_yq_7['订单号'].count()
    # 逾期15+订单
    df_yq_15= df4_dq[(df4_dq['逾期天数']>=15)]
    dd_yq_15 = df_yq_15['订单号'].count()
    # 逾期30+订单
    df_yq_30= df4_dq[(df4_dq['逾期天数']>=30)]
    dd_yq_30 = df_yq_30['订单号'].count()

    # 今日到期订单
    df_dq_today = df4_dq[df4_dq['预计还款日期']==td]
    dd_dq_today_count = df_dq_today['订单号'].count()
    # 今日到期已还款
    dq_yhk = df_dq_today[df_dq_today['实付金额'] >= df_dq_today['应付金额']]
    dd_dq_yhk = dq_yhk['订单号'].count()
    # 今日到期未还款
    dq_whk = df_dq_today[df_dq_today['实付金额'] < df_dq_today['应付金额']]
    dd_dq_whk = dq_whk['订单号'].count()
    dd_dq_whk

    # 汇总
    dd_all = pd.DataFrame({
    '截止日期': [td],
    '本月到期订单': [dd_dq_month],
    '今日到期订单': [dd_dq_today_count],
    '逾期1+订单': [dd_yq_1],
    '逾期4+订单': [dd_yq_4],
    '逾期7+订单': [dd_yq_7],
    '逾期15+订单': [dd_yq_15],
    '逾期30+订单': [dd_yq_30],
    '今日到期已还款订单': [dd_dq_yhk],
    '今日到期未还款订单': [dd_dq_whk]
    })
    dd_all['本月1+逾期率'] = dd_yq_1/dd_dq_month * 100
    dd_all['本月1+逾期率'] = dd_all['本月1+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月4+逾期率'] = dd_yq_4/dd_dq_month * 100
    dd_all['本月4+逾期率'] = dd_all['本月4+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月7+逾期率'] = dd_yq_7/dd_dq_month * 100
    dd_all['本月7+逾期率'] = dd_all['本月7+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月15+逾期率'] = dd_yq_15/dd_dq_month * 100
    dd_all['本月15+逾期率'] = dd_all['本月15+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['本月30+逾期率'] = dd_yq_30/dd_dq_month * 100
    dd_all['本月30+逾期率'] = dd_all['本月30+逾期率'].apply(lambda x: f'{x: .2f}%')
    dd_all['今日到期已还款率'] = dd_dq_yhk/dd_dq_today_count * 100
    dd_all['今日到期已还款率'] = dd_all['今日到期已还款率'].apply(lambda x: f'{x:.2f}%')
    dd_all['今日到期未还款率'] = dd_dq_whk/dd_dq_today_count * 100
    dd_all['今日到期未还款率'] = dd_all['今日到期未还款率'].apply(lambda x: f'{x:.2f}%')
    dd_all = dd_all[['截止日期','本月到期订单','逾期1+订单','逾期4+订单','逾期7+订单','逾期15+订单','逾期30+订单','本月1+逾期率','本月4+逾期率','本月7+逾期率','本月15+逾期率','本月30+逾期率','今日到期订单','今日到期已还款订单','今日到期已还款率','今日到期未还款订单','今日到期未还款率']]
    dd_all

    return df4_dq, dd_all, df_yq_1, df_yq_4, df_yq_7, df_yq_15, df_dq_today, dq_yhk, dq_whk


## 历史逾期

In [80]:
# df4_history = df4[df4['预计还款月份']=='2024-10']
# df4_history[['下单日期', '预计还款日期', '实际还款日期', '逾期天数']]
# df4_history[df4_history['逾期天数']>=1][['下单日期', '预计还款日期', '实际还款日期', '逾期天数']]

In [81]:
def qd_info(df, qd, qd_name):    
    df = df[df[qd]==qd_name]
    df_count = df['订单号'].count()
    df_jj = df[df['是否进件']==1]
    df_jj = df_jj[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '应付金额', '实付金额']]
    df_jj_count = df_jj['订单号'].count()

    df_yq = df[df['逾期天数']>=1]
    df_yq = df_yq[['订单号', '姓名', '手机号', '产品名称', '下单日期', '预计还款日期', '实际还款日期', '逾期天数', '应付金额', '实付金额']]
    df_yq_count = df_yq['订单号'].count()
    df_yq
    return df_jj, df_yq

In [82]:
current_date = pd.Timestamp.now()
before_date = current_date - pd.DateOffset(months=1)
before_date2 = current_date - pd.DateOffset(months=3)
date1 = before_date.strftime('%Y-%m')
date2 = current_date.strftime('%Y-%m')
# date2 = before_date2.strftime('%Y-%m')
df4_dq, dd_all, df_yq_1, df_yq_4, df_yq_7, df_yq_15, df_dq_today, dq_yhk, dq_whk = order_end(df4, date1, date2, '首逾')
dd_all

,截止日期,本月到期订单,逾期1+订单,逾期4+订单,逾期7+订单,逾期15+订单,逾期30+订单,本月1+逾期率,本月4+逾期率,本月7+逾期率,本月15+逾期率,本月30+逾期率,今日到期订单,今日到期已还款订单,今日到期已还款率,今日到期未还款订单,今日到期未还款率
0,2025-03-17,674,51,22,8,0,0,7.57%,3.26%,1.19%,0.00%,0.00%,80,49,61.25%,31,38.75%


In [83]:
def fpd(df, keys):
    # 逾期1+订单
    df_yq_1= df[(df['逾期天数']>=1)]
    dd_yq_1_count = df_yq_1['订单号'].count()
    dd_yq_1_sum = df_yq_1['应付金额'].sum()
    # 逾期4+订单
    df_yq_4= df[(df['逾期天数']>=4)]
    dd_yq_4_count = df_yq_4['订单号'].count()
    dd_yq_4_sum = df_yq_4['应付金额'].sum()
    # 逾期7+订单
    df_yq_7= df[(df['逾期天数']>=7)]
    dd_yq_7_count = df_yq_7['订单号'].count()
    dd_yq_7_sum = df_yq_7['应付金额'].sum()
    # 逾期15+订单
    df_yq_15= df[(df['逾期天数']>=15)]
    dd_yq_15_count = df_yq_15['订单号'].count()
    dd_yq_15_sum = df_yq_15['应付金额'].sum()
    # 逾期30+订单
    df_yq_30= df[(df['逾期天数']>=30)]
    dd_yq_30_count = df_yq_30['订单号'].count()
    dd_yq_30_sum = df_yq_30['应付金额'].sum()

    df_fpd = pd.DataFrame({
        'fpd1+_count': [dd_yq_1_count],
        'fpd4+_count': [dd_yq_4_count],
        'fpd7+_count': [dd_yq_7_count],
        'fpd15+_count': [dd_yq_15_count],
        'fpd30+_count': [dd_yq_30_count]
    })

    df_cpd = pd.DataFrame({
        'cpd1+_count': [dd_yq_1_count],
        'cpd4+_count': [dd_yq_4_count],
        'cpd7+_count': [dd_yq_7_count],
        'cpd15+_count': [dd_yq_15_count],
        'cpd30+_count': [dd_yq_30_count]
    })

    if keys == 'fpd':
        return df_yq_1, df_yq_4, df_yq_7, df_yq_15, df_yq_30, df_fpd
    elif keys == 'cpd':
        return df_yq_1, df_yq_4, df_yq_7, df_yq_15, df_yq_30, df_cpd

## 芝麻租物

In [84]:
df4_zm_jj, df4_zm_yq = qd_info(df4_dq, '归属渠道', '芝麻租物')
# df4_zm_yq

In [85]:
df_zmyq_1, df_zmyq_4, df_zmyq_7, df_zmyq_15, df_zmyq_30, df_zm_fpd = fpd(df4_zm_yq, 'fpd')
df_zm_fpd

,fpd1+_count,fpd4+_count,fpd7+_count,fpd15+_count,fpd30+_count
0,24,8,5,0,0


## 搜索渠道

In [86]:
df4_ss_jj, df4_ss_yq = qd_info(df4_dq, '归属渠道', '搜索渠道')
df_ssyq_1, df_ssyq_4, df_ssyq_7, df_ssyq_15, df_ssyq_30, df_ss_fpd = fpd(df4_ss_yq, 'fpd')
df4_ss_yq

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,逾期天数,应付金额,实付金额
181213,A2025020311193017,张哲群,15564467981,首月1元 iPhone16ProMax 全新未激活 赠送壳膜 顺丰包邮,2025-02-03 11:19:31,2025-03-06,NaT,11.0,666.32,0.00
181343,A20250205175847247,梁超文,15992602367,iPhone 16 Pro Max 全新未激活 国行正品 顺丰包邮,2025-02-05 17:58:48,2025-03-08,2025-03-15 02:11:59,7.0,716.46,716.46
181583,A2025020810134051,崔畅,13102718898,首月1元 iPhone16ProMax 全新未激活 赠送壳膜 顺丰包邮,2025-02-08 10:13:41,2025-03-11,2025-03-12 02:32:43,1.0,1082.94,1082.94
181665,A20250208145824302,林培杰,13615930510,iPhone 16 Pro 全新未激活 国行正品 寄出包邮,2025-02-08 14:58:25,2025-03-11,2025-03-13 18:14:12,2.0,974.62,974.62
181677,A20250208153818250,刘壮,13399106567,iPhone 16 Pro 全新未激活 国行正品 寄出包邮,2025-02-08 15:38:19,2025-03-11,2025-03-14 03:10:47,3.0,573.14,573.14
181741,A20250209122933102,刘娜,13940784317,iPhone 16 Pro Max 全新未激活 国行正品 顺丰包邮,2025-02-09 12:29:33,2025-03-12,NaT,5.0,716.46,0.00
181771,A20250209154219165,张峰浩,13372931086,iPhone 16 Pro 全新未激活 国行正品 寄出包邮,2025-02-09 15:42:19,2025-03-12,NaT,5.0,644.82,0.00
181777,A20250209164038174,陈应照,18952559199,首月1元 iPhone16ProMax 全新未激活 赠送壳膜 顺丰包邮,2025-02-09 16:40:38,2025-03-12,2025-03-17 13:38:56,5.0,666.32,666.32
181801,A20250208184954154,樊又源,18235991576,iPhone 16 Pro Max 全新未激活 国行正品 顺丰包邮,2025-02-08 18:49:54,2025-03-11,NaT,6.0,1082.94,0.00
181823,A2025021010055020,范丹丹,19307080929,iPhone 16 Pro Max 全新未激活 国行正品 顺丰包邮,2025-02-10 10:05:50,2025-03-13,NaT,4.0,1082.94,0.00


# 单人会话

In [87]:
df4_dr_jj, df4_dr_yq = qd_info(df4_dq, '来源渠道', '单人聊天会话中的小程序消息卡片（分享）')
df4_dr_yq

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,逾期天数,应付金额,实付金额
181499,A20250207123811131,周二平,18255165173,iPhone 16 Pro Max 全新未激活 国行正品 顺丰包邮,2025-02-07 12:38:11,2025-03-10,NaT,7.0,1082.94,0.0


## 八派信息

### 逾期订单

In [88]:
# df4_bp_jj, df4_bp_yq = qd_info(df4_dq, '归属渠道', '八派信息')
# df4_bp_yq
# df4_bp_jj

In [89]:
# df4_bp_jj_history, df4_bp_yq_history = qd_info(df4_dq_history, '归属渠道', '八派信息')
# df4_bp_yq_history

In [90]:
df_dryq_1, df_dryq_4, df_dryq_7, df_dryq_15, df_dryq_30, df_fpd = fpd(df4_dr_yq, 'fpd')
df_fpd

,fpd1+_count,fpd4+_count,fpd7+_count,fpd15+_count,fpd30+_count
0,1,1,1,0,0


### 八派信息FPD

In [91]:
# df_bpyq_1, df_bpyq_4, df_bpyq_7, df_bpyq_15, df_bpyq_30, df_fpd = fpd(df4_bp_yq, 'fpd')

### 八派信息CPD

In [92]:
# df4_bp_yq_cpd = df4_bp_yq[df4_bp_yq['实际还款日期'].isna()]
# df_bpyq_1, df_bpyq_4, df_bpyq_7, df_bpyq_15, df_bpyq_30, df_cpd = fpd(df4_bp_yq_cpd, 'cpd')

## 九州信息

In [93]:
df4_jz_jj, df4_jz_yq = qd_info(df4_dq, '归属渠道', '九州信息')
df4_jz_yq
# df4_jz_jj

,订单号,姓名,手机号,产品名称,下单日期,预计还款日期,实际还款日期,逾期天数,应付金额,实付金额


In [94]:
# df4_jz_jj_history, df4_jz_yq_history = qd_info(df4_dq_history, '归属渠道', '九州信息')
# df4_jz_yq_history

In [95]:
d1 = date1.split('-')[-1]
d2 = date2.split('-')[-1]
# bp_order = df4_bp_jj['订单号'].count()
# bp_money = df4_bp_jj['应付金额'].sum()
# bp_order_history = df4_bp_jj_history['订单号'].count()
jz_order = df4_jz_jj['订单号'].count()
dr_order = df4_dr_jj['订单号'].count()
# jz_order_history = df4_jz_jj_history['订单号'].count()

In [96]:
zm_order = df4_zm_jj['订单号'].count()

In [97]:
ss_order = df4_ss_jj['订单号'].count()

In [98]:
df4_jz_yq['实际还款日期'].isna()

Series([], Name: 实际还款日期, dtype: bool)

## 信息汇总

In [99]:
def info(df):
    wh, yh = 0, 0
    for index, row in df.iterrows():
        if pd.isna(row['实际还款日期']):
            wh += 1
        else:
            yh += 1
    return wh, yh
# bp_wh,bp_yh = info(df4_bp_yq)
jz_wh, jz_yh = info(df4_jz_yq)
dr_wh, dr_yh = info(df4_dr_yq)
# bp_yq_rate = round(((bp_wh+bp_yh) / bp_order) * 100, 2)
# bp_yq_t_rate = round((bp_wh / bp_order) * 100, 2)

jz_yq_rate = round(((jz_wh+jz_yh) / jz_order) * 100, 2)
jz_yq_t_rate = round((jz_wh / jz_order) * 100, 2)
dr_yq_rate = round(((dr_wh+dr_yh) / dr_order) * 100, 2)
dr_yq_t_rate = round((dr_wh / dr_order) * 100, 2)
all_str = f'下单时间为{d1}月的订单，单人会话{d2}月到期有：{dr_order}笔，首逾{dr_wh+dr_yh}笔，{dr_wh}笔未还，{dr_yh}笔已还，首逾率{dr_yq_rate}%， 当前逾期率{dr_yq_t_rate}%;九州信息到期{jz_order}笔，首逾{jz_wh+jz_yh}笔，{jz_wh}笔未还，{jz_yh}笔已还，首逾率{jz_yq_rate}%， 当前逾期率{jz_yq_t_rate}%。'
all_str


'下单时间为02月的订单，单人会话03月到期有：6笔，首逾1笔，1笔未还，0笔已还，首逾率16.67%， 当前逾期率16.67%;九州信息到期4笔，首逾0笔，0笔未还，0笔已还，首逾率0.0%， 当前逾期率0.0%。'

In [100]:
# all_str2 = f'下单时间为{d1}月的订单，八派{d2}月到期有：{bp_order}笔，首逾{bp_wh+bp_yh}笔，{bp_wh}笔未还，{bp_yh}笔已还;九州信息到期{jz_order}笔，首逾{jz_wh+jz_yh}笔，{jz_wh}笔未还，{jz_yh}笔已还。'
# all_str2

In [101]:
# bp_wh, jz_wh, bp_yh, jz_yh = info(df4_bp_yq_history, df4_jz_yq_history)

# bp_wh_h,bp_yh_h = info(df4_bp_yq_history)
# jz_wh_h, jz_yh_h = info(df4_jz_yq_history)
# bp_yq_rate_h = round(((bp_wh_h+bp_yh_h) / bp_order_history) * 100, 2)
# bp_yq_t_rate_h = round((bp_wh_h / bp_order_history) * 100, 2)

# jz_yq_rate_h = round(((jz_wh+jz_yh) / jz_order_history) * 100, 2)
# jz_yq_t_rate_h = round((jz_wh_h / jz_order_history) * 100, 2)
# all_str_h = f'下单时间为{d1}月的订单，八派{d2}月到期有：{bp_order_history}笔，首逾{bp_wh_h+bp_yh_h}笔，{bp_wh_h}笔未还，{bp_yh_h}笔已还，首逾率{bp_yq_rate_h}%， 当前逾期率{bp_yq_t_rate_h}%;九州信息到期{jz_order_history}笔，首逾{jz_wh_h+jz_yh_h}笔，{jz_wh_h}笔未还，{jz_yh_h}笔已还，首逾率{jz_yq_rate_h}%， 当前逾期率{jz_yq_t_rate_h}%。'
# all_str_h

In [102]:
def rate(df, order_all, keys):
    # 到期日期
    df['下单月份'] = date1
    # 总订单
    df['到期笔数'] = order_all
    # fpd1+
    df[f'{keys}1+'] = df[f'{keys}1+_count'] / order_all
    df[f'{keys}1+'] = df[f'{keys}1+'].apply(lambda x: format(x, '.2%'))
    # fpd4+
    df[f'{keys}4+'] = df[f'{keys}4+_count'] / order_all
    df[f'{keys}4+'] = df[f'{keys}4+'].apply(lambda x: format(x, '.2%'))
    # fpd7+
    df[f'{keys}7+'] = df[f'{keys}7+_count'] / order_all
    df[f'{keys}7+'] = df[f'{keys}7+'].apply(lambda x: format(x, '.2%'))
    # fpd15+
    df[f'{keys}15+'] = df[f'{keys}15+_count'] / order_all
    df[f'{keys}15+'] = df[f'{keys}15+'].apply(lambda x: format(x, '.2%'))
    # fpd30+
    df[f'{keys}30+'] = df[f'{keys}30+_count'] / order_all
    df[f'{keys}30+'] = df[f'{keys}30+'].apply(lambda x: format(x, '.2%'))

    return df


### fpd_rate

In [103]:
# df__bp_fpd = rate(df_fpd, bp_order, 'fpd')
# df__bp_fpd = df__bp_fpd[['下单月份', '到期笔数', 'fpd1+_count', 'fpd4+_count', 'fpd7+_count', 'fpd15+_count', 'fpd30+_count', 'fpd1+', 'fpd4+', 'fpd7+', 'fpd15+', 'fpd30+']]
# df__bp_fpd

In [104]:
df_zm_fpd = rate(df_zm_fpd, zm_order, 'fpd')
df_zm_fpd = df_zm_fpd[['下单月份', '到期笔数', 'fpd1+_count', 'fpd4+_count', 'fpd7+_count', 'fpd15+_count', 'fpd30+_count', 'fpd1+', 'fpd4+', 'fpd7+', 'fpd15+', 'fpd30+']]
df_zm_fpd

,下单月份,到期笔数,fpd1+_count,fpd4+_count,fpd7+_count,fpd15+_count,fpd30+_count,fpd1+,fpd4+,fpd7+,fpd15+,fpd30+
0,2025-02,317,24,8,5,0,0,7.57%,2.52%,1.58%,0.00%,0.00%


In [105]:
df_ss_fpd = rate(df_ss_fpd, ss_order, 'fpd')
df_ss_fpd = df_ss_fpd[['下单月份', '到期笔数', 'fpd1+_count', 'fpd4+_count', 'fpd7+_count', 'fpd15+_count', 'fpd30+_count', 'fpd1+', 'fpd4+', 'fpd7+', 'fpd15+', 'fpd30+']]
df_ss_fpd

,下单月份,到期笔数,fpd1+_count,fpd4+_count,fpd7+_count,fpd15+_count,fpd30+_count,fpd1+,fpd4+,fpd7+,fpd15+,fpd30+
0,2025-02,303,22,11,2,0,0,7.26%,3.63%,0.66%,0.00%,0.00%


### cpd_rate

In [106]:
# df__bp_cpd = rate(df_cpd, bp_order, 'cpd')
# df__bp_cpd = df__bp_cpd[['下单月份', '到期笔数', 'cpd1+_count', 'cpd4+_count', 'cpd7+_count', 'cpd15+_count', 'cpd30+_count', 'cpd1+', 'cpd4+', 'cpd7+', 'cpd15+', 'cpd30+']]
# df__bp_cpd

# 设置定时任务

In [109]:
# from apscheduler.schedulers.blocking import BlockingScheduler
# from apscheduler.schedulers.background import BackgroundScheduler
# 
# # hour = 9
# # minute = 10
# # week_name = pd.Timestamp.now().dayofweek
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# def my_job1():
#     # print(f'执行定时任务：现在是每日的{hour}点{minute}分')
#     print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = 'F:/需求/到期订单/'
#     path2 = 'F:/需求/截止本月到期订单/'
#     path3 = 'F:/需求/八派、九州/'
#     # 到期提醒
#     with pd.ExcelWriter(path1 + f'到期订单_信审_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#         df_xs.to_excel(writer, sheet_name=now_date, index=False)
#     # with pd.ExcelWriter(path1 + f'到期订单_催收_{now_date}-{next_date}.xlsx', engine='openpyxl') as writer:
#     #     df_cs.to_excel(writer, sheet_name=now_date, index=False)
#     # 本月到期订单与逾期订单
#     # with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl') as writer:
#     #     dd_all.to_excel(writer, sheet_name='订单汇总', index=False)
#     # with pd.ExcelWriter(path2 + f'本月到期订单截止_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     #     df4_dq.to_excel(writer, sheet_name='本月到期订单', index=False)
#     #     df_dq_today.to_excel(writer, sheet_name='今日到期订单', index=False)
#     #     df_yq_1.to_excel(writer, sheet_name='逾期1+订单', index=False)
#     #     df_yq_4.to_excel(writer, sheet_name='逾期4+订单', index=False)
#     #     df_yq_7.to_excel(writer, sheet_name='逾期7+订单', index=False)
#     #     df_yq_15.to_excel(writer, sheet_name='逾期15+订单', index=False)
#     #     dq_yhk.to_excel(writer, sheet_name='今日到期已还款订单', index=False)
#     #     dq_whk.to_excel(writer, sheet_name='今日到期未还款订单', index=False)
#     # 渠道逾期数据
#     # with pd.ExcelWriter(path3 + f'进件逾期数据_{Today1}.xlsx', engine='openpyxl') as writer:
#         # df4_bp_jj.to_excel(writer, sheet_name='八派信息进件数据', index=False)
#     # with pd.ExcelWriter(path3 + f'进件逾期数据_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#         # df4_bp_yq.to_excel(writer, sheet_name='八派信息逾期数据', index=False)
#         # df4_jz_jj.to_excel(writer, sheet_name='九州信息进件数据', index=False)
#         # df4_jz_yq.to_excel(writer, sheet_name='九州信息逾期数据', index=False)
#         # df__bp_fpd.to_excel(writer, sheet_name='八派信息fpd', index=False)
#         # df__bp_cpd.to_excel(writer, sheet_name='八派信息cpd', index=False)
# scheduler1 = BackgroundScheduler()
# # 添加一个cron任务，每天的16点55分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
# 
# # 启动scheduler1
# scheduler1.start()

# 到期订单（方全龙需求）

In [54]:
df5.loc[:,"来源渠道"]=df5["channel_name"].fillna("未知渠道")

In [55]:

df5.loc[:,"归属渠道"]=df5.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['merchant_type_id'], x['order_type']),axis=1)

KeyError: 'merchant_type_id'

In [52]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")
dfck = pd.merge(df_ck[['订单号', '采购金额']],df5,left_on="订单号", right_on="订单号", how='inner')
dfck = dfck[dfck.采购金额>0]
dfck.loc[:, '毛利'] = ((dfck.买断价+dfck.溢价费-dfck.采购金额)/dfck.采购金额).map(lambda x: format(x, '.2%'))

In [53]:
dfck_new = dfck.copy()
def chennel(dfck_new):
    dfck_new.loc[:, '内存'] = dfck_new.规格.str.split('：').str[1].str.split(' ').str[0]
    dfck_new.loc[:, '型号内存'] = dfck_new.型号+ '_' + dfck_new.内存
    dfck_new_group = dfck_new.groupby('型号内存').agg({'订单号': 'count', '采购金额': 'sum', '总应收': 'sum', '买断价': 'sum', '溢价费': 'sum', '总租金': 'sum', '总押金': 'sum', '买断尾款': 'sum', '残值': 'sum'}).rename(columns={'订单号': '数量'})
    dfck_new_group.loc[:, '毛利'] = ((dfck_new_group.买断价+dfck_new_group.溢价费-dfck_new_group.采购金额)/dfck_new_group.采购金额).map(lambda x: format(x, '.2%'))
    dfck_new_group.loc[:, '租售比'] = ((dfck_new_group.总租金/dfck_new_group.数量)/(dfck_new_group.总押金/dfck_new_group.数量)).map(lambda x: format(x, '.2%'))
    return dfck_new_group

In [54]:
dfck_new_zw = dfck_new[dfck_new.归属渠道=='芝麻租物']
dfck_new_group = chennel(dfck_new_zw)

In [55]:
dfck_new_fzw = dfck_new[dfck_new.归属渠道!='芝麻租物']
dfck_new_fgroup = chennel(dfck_new_fzw)

In [56]:

with pd.ExcelWriter(f'F:/需求/方全龙需求/订单_{dd_date}.xlsx', engine='openpyxl') as writer:
    dfck.to_excel(writer, sheet_name=f'订单_{dd_date}', index=False)
    dfck_new_group.to_excel(writer, sheet_name=f'芝麻租物订单SKU_{dd_date}')
    dfck_new_fgroup.to_excel(writer, sheet_name=f'非芝麻租物订单SKU_{dd_date}')